### CMSC 673 Intro to NLP Project
#### Patty Delafuente and Neel Patel

This is an example of a downstream task that you could use to interface a trp classifier with a Llama 2 chat model.
This example uses the trained model classifier to predict input. 
Adapted from https://python.langchain.com/docs/modules/agents/how_to/custom_multi_action_agent


In [3]:
#this set ups initial libraries, classes and tools and initializes llm chat model
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

from langchain.chains import TransformChain, SequentialChain, LLMChain
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import PromptTemplate
from langchain.agents import BaseSingleActionAgent
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain.llms import BaseLLM

from typing import List, Tuple, Any, Union, Optional
from pydantic import root_validator, Field
from abc import abstractmethod
    
model_kwargs = {"do_sample": True, "temperature": 0.4, "max_length": 4096}

model_name = "models/meta/Llama-2-7b-chat-hf"
llama_pipe = pipeline("text-generation", model=model_name, device_map="auto", model_kwargs=model_kwargs);

llm = HuggingFacePipeline(pipeline=llama_pipe)



class SetParams:
    def __init__(self, my_llm, **new_params):
        self.pipeline = my_llm.pipeline
        self._old_params = {**self.pipeline._forward_params}
        self._new_params = new_params
    
    def __enter__(self):
        self.pipeline._forward_params.update(**self._new_params)

    def __exit__(self ,type, value, traceback):
        for k in self._new_params.keys(): 
            del self.pipeline._forward_params[k]
        self.pipeline._forward_params.update(self._old_params)
        

from io import StringIO
import sys
from typing import Dict, Optional

from langchain.agents.tools import Tool


## General recipe for making new tools. 

class AutoTool:

    """Keep-Reasoning Tool
    
    This is an example tool. The input will be returned as the output
    """
    
    def get_tool(self, **kwargs):
        ## Shows also how some open-source libraries like to support auto-variables
        doc_lines = self.__class__.__doc__.split('\n')
        class_name = doc_lines[0]                     ## First line from the documentation
        class_desc = "\n".join(doc_lines[1:]).strip() ## Essentially, all other text
        
        return Tool(
            name        = kwargs.get('name',        class_name),
            description = kwargs.get('description', class_desc),
            func        = kwargs.get('func',        self.run),
        )
    
    def run(self, command: str) -> str:
        ## The function that should be ran to execute the tool forward pass
        return command
    
class AskForInputTool(AutoTool):

    """Ask-For-Input Tool
    
    This tool asks the user for input, which you can use to gather more information. 
    Use only when necessary, since their time is important and you want to give them a great experience! For example:
    Action-Input: What is your name?
    """
    
    def __init__(self, fn = input):
        self.fn = fn
    
    def run(self, command: str) -> str:
        response = self.fn(command)
        return response

from langchain.agents import load_tools
from langchain.agents import initialize_agent


tools = [
    AutoTool().get_tool(),
    AskForInputTool().get_tool()
]
agent_executor = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True,
    agent_kwargs = dict(
        prefix="<s>[INST]<<SYS>>",
        suffix="[/INST]\nQuestion: {input}\n\nThought:{agent_scratchpad}",
    )
)


class MyAgentBase(BaseSingleActionAgent):
    
    
        
    @root_validator
    def validate_input(cls, values: Any) -> Any:
        '''
        Think of this like the BaseModel's __init__ method
        You'll see how it works in the stencil, but this is where components get initialized
        '''
        return values
    
    @abstractmethod
    def plan(self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any): 
        '''
        Taking the "intermediate_steps" as the history of steps.
        Decide on the next action to take! Return the required action 
        (returns a query from the action method)
        '''
        pass

    def action(self, tool, tool_input, finish=False) -> Union[AgentAction, AgentFinish]:
        '''Takes the action associated with the tool and feeds it the necessary parameters'''
        if finish: return AgentFinish({"output": tool_input},           log = f"\nFinal Answer: {tool_input}\n")
        else:      return AgentAction(tool=tool, tool_input=tool_input, log = f"\nAgent: {tool_input.strip()}\n")
        # else:    return AgentAction(tool=tool, tool_input=tool_input, log = f"\nTool: {tool}\nInput: {tool_input}\n") ## Actually Correct
    
    async def aplan(self, intermediate_steps, **kwargs):
        '''The async version of plan. It has to be defined because abstractmethod'''
        return await self.plan(intermediate_steps, **kwargs)
    
    @property
    def input_keys(self):
        return ["input"]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#this is the inferencing interface that enables user input but also has a pipeline to the trp classification model
llama_full_prompt = PromptTemplate.from_template(
    template="<s>[INST]<<SYS>>{sys_msg}<</SYS>>\n\nContext:\n{history}\n\nHuman: {input}\n[/INST] {primer}",
)

llama_prompt = llama_full_prompt.partial(
    sys_msg = ( 
        "You are a helpful, respectful and honest AI assistant."
        "\nAlways answer as helpfully as possible, while being safe."
        "\nPlease be brief and efficient unless asked to elaborate, and follow the conversation flow."
    ),
    primer = "",
    history = "",
)


from langchain.chains import ConversationChain
checkpoint_dir = 'roberta-large-lora-token-classification/checkpoint-1690/'
trp_pipe = pipeline("text-classification", model=checkpoint_dir)

from langchain.memory import ConversationBufferMemory ##pd
memory = ConversationBufferMemory(memory_key="history")

class MyAgent(MyAgentBase):
    
    ## Instance methods that can be passed in as BaseModel arguments. 
    ## Will be associated with self
    
    general_prompt : PromptTemplate
    llm            : BaseLLM
    
    general_chain  : Optional[LLMChain]
    max_messages   : int                   = Field(7, gt=1)
    
    temperature    : float                 = Field(0.6, gt=0, le=1)
    max_new_tokens : int                   = Field(128, ge=1, le=2048)
    eos_token_id   : Union[int, List[int]] = Field(2, ge=0)
    gen_kw_keys = ['temperature', 'max_new_tokens', 'eos_token_id']
    gen_kw = {}
    
    trp_prediction  : float = 0.5
    
    
    
    @root_validator
    def validate_input(cls, values: Any) -> Any:
        '''Think of this like the BaseModel's __init__ method'''
        if not values.get('general_chain'):
            llm = values.get('llm')
            prompt = values.get("general_prompt")
            values['general_chain'] = LLMChain(llm=llm, prompt=prompt,memory=memory)  
        values['gen_kw'] = {k:v for k,v in values.items() if k in values.get('gen_kw_keys')}
        return values


    def plan(self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any): 
        '''Takes in previous logic and generates the next action to take!'''
        
 
        tool, response = "Ask-For-Input Tool", "Hi! How can I help you?"
        if len(intermediate_steps) == 0:
            return self.action(tool, response)
        
        ## History of past agent queries/observations
        queries      = [step[0].tool_input for step in intermediate_steps]
        observations = [step[1]            for step in intermediate_steps]
        last_obs     = observations[-1]    # Most recent observation (i.e. user input)


        if len(observations) >= self.max_messages:
            response = "Thanks so much for the chat, and hope to see ya later! Goodbye!"
            return self.action(tool, response, finish=True)

        with SetParams(llm, **self.gen_kw):
            self.trp_prediction = trp_pipe(last_obs)
            
            print("TRP prediction for input is: ",self.trp_prediction, " We could also intercept the response and do an action.")
            
                       
            response = self.general_chain.run(last_obs)

        
        ## [Default Case] Send over the response back to the user and get their input!
        return self.action(tool, response)
    

    def reset(self):
        self.trp_prediction = 0
        
        if getattr(self.general_chain, 'memory', None) is not None:
            self.general_chain.memory.clear()  ## Hint about what general_chain should be...


####################################################################################
## Define how you want your conversation to go. You can also use your own input
## The below example in conversation_gen exercises some of the requirements.

## pd - this part is needed to pass in history as input
llama_prompt.input_variables = ['input', 'history'] ##pd
student_name = "Patty"   ## TODO: What's your name

ask_via_input = False       ## TODO: When you're happy, try supplying your own inputs
def predict_trp(text):
    emotion = emo_pipe(text)[0]['label']
    return emotion
        
def predict_trp(text):
    trp_score = trp_pipe(text)[0]['score']
    #toxic_score = float(format(toxic_score, '.2f'))
    
    return toxic_score
    
def conversation_gen():
    yield f"Hello! How's it going? My name is {student_name}! Nice to meet you!"
    yield "Can you tell me about Christmas"
    yield "What is my name?"          
    yield "Goodbye!"                                       
    raise KeyboardInterrupt()

conversation_instance = conversation_gen()
converser = lambda x: next(conversation_instance) 

if ask_via_input:
    converser = input  ## Alternatively, supply your own inputs

agent_kw = dict(
    llm = llm,
    general_prompt = llama_prompt, ##pd
    max_new_tokens = 128,
    eos_token_id = [2]   
)

agent_ex = AgentExecutor.from_agent_and_tools(
    agent = MyAgent(**agent_kw),
    tools=[AskForInputTool(converser).get_tool()], 
    verbose=True,
    memory=memory ##pd
)


try: agent_ex.run("")
except KeyboardInterrupt: print("KeyboardInterrupt")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




> Entering new AgentExecutor chain...

Agent: Hi! How can I help you?
Hello! How's it going? My name is Patty! Nice to meet you!TRP prediction for input is:  [{'label': 'LABEL_1', 'score': 0.5108490586280823}]  We could also intercept the response and do an action.

Agent: Hello Patty! *smiling* It's nice to meet you too! I'm just an AI, here to help you with any questions or tasks you may have. How can I assist you today? 😊
Can you tell me about ChristmasTRP prediction for input is:  [{'label': 'LABEL_1', 'score': 0.5099822878837585}]  We could also intercept the response and do an action.

Agent: Of course, Patty! *smiling* Christmas is a holiday that is celebrated by many cultures around the world, typically on December 25th. It commemorates the birth of Jesus Christ and is observed by Christians as a time of gift-giving, family gatherings, and spiritual reflection.

There are many traditions and customs associated with Christmas, such as decorating trees, hanging stockings, and s